<a href="https://colab.research.google.com/github/jaeyoung-jae-park/Joint-model-assisted-Decision-Rule/blob/main/Numerical_experiments_FashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data preparation

## Data Loading

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

(X, y), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X = X/255
X = np.array(X).reshape((60000, 28,28,1))

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
pd.DataFrame(np.repeat(np.arange(10), 6000)).value_counts()

9    6000
8    6000
7    6000
6    6000
5    6000
4    6000
3    6000
2    6000
1    6000
0    6000
dtype: int64

In [ ]:
X.shape

(60000, 28, 28, 1)

## Label setting
Target - Shirt

Auxiliary outcomes - t-shirt/top/shirt and coat/shirt

In [ ]:
y_aux = np.vstack(((y == 0)*1 + (y == 6)*1, (y == 4)*1 + (y==6)*1))
y_tgt = (y == 6) * 1
y_aux_agg = np.transpose(np.vstack((y_tgt, y_aux, y_tgt*y_aux[0,:], y_tgt*y_aux[1,:])))

In [ ]:
np.sum(y_aux_agg, axis=0)

array([ 6000, 12000, 12000,  6000,  6000])

# Hyperparameter settings

Neural network structures are defined for the following seven models: Baseline, NN, NN-joint, LDR-NN-joint, LDR-CIDNN, NLDR-NN-joint, and NLDR-CIDNN.

The optimizer is stochastic gradient descent with a learning rate of $10^{-3}$ and a momentum of 0.9. 
For each training set (9 folds), we further split it and use 20\% of the data as the validation set to avoid overfitting. 
The learning rate will decrease and the training may stop early based on the validation loss values. 
When a validation loss value does not achieve one smaller than the minimum of the last 5 epochs, the learning rate reduces to one tenth of its previous value. Further, although the total number of epochs is 300, the training will stop early, if a loss value smaller than the minimum is not obtained within 10 epochs.

## Baseline

A linear decision rule does not have hidden layers. The network only consists of an input layer and an output layer.



In [ ]:
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, concatenate
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import SGD
import tensorflow as tf

In [ ]:
def define_Baseline(output_dim):
  output_dim = output_dim

  model = Sequential()
  model.add(Flatten(input_shape = (28, 28, 1)))
  model.add(Dense(output_dim, activation = "sigmoid"))
  model.summary()

  return model

def train_Baseline(model, X, y, epochs, batch_size):
  if model is None:
    if len(y.shape) == 1:
      model = define_Baseline(1)
    else: model= define_Baseline(y.shape[1])
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
  rl = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

  model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate = 0.001, momentum=0.9), metrics=['accuracy'])
  history = model.fit(X , y, batch_size = batch_size, epochs = epochs, validation_split=0.2, callbacks=[es, rl], verbose=2)
  return model, history


## NN and NN-joint

The network stacks two pairs of convolution and max pooling, a ﬂatten layer, a dropout layer with a rate of 0.5, and a fully-connected hidden layer containing 512 units. The ﬁrst pair includes a convolution layer with 64 ﬁlters and a kernel size of 7-by-7, and a max pooling layer with a size of 2-by-2. The second pair includes a convolution layer with 128 ﬁlters and a kernel size of 5-by-5, and a max pooling layer with a size of 2-by-2.

In [ ]:
def define_NN(output_dim):
  output_dim = output_dim

  model = Sequential()
  model.add(Conv2D(64, 7, input_shape = (28, 28, 1), padding='same', activation = "relu"))
  model.add(MaxPooling2D(pool_size = (2,2)))
  model.add(Conv2D(128, 5, activation = "relu", padding='same'))
  model.add(MaxPooling2D(pool_size = (2,2)))
  
  model.add(Flatten())
  model.add(Dropout(0.5))
  model.add(Dense(512, activation="relu"))
  model.add(Dense(output_dim, activation = "sigmoid"))
  model.summary()

  return model

def train_NN(model, X, y, epochs, batch_size):
  if model is None:
    if len(y.shape) == 1:
      model = define_NN(1)
    else: model= define_NN(y.shape[1])
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
  rl = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)


  model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate = 0.001, momentum=0.9), metrics=['accuracy'])
  history = model.fit(X , y, batch_size = batch_size, epochs = epochs, validation_split=0.2, callbacks=[es, rl], verbose=2)
  
  return model, history


## CIDNN

To calculate CIScore correctly, the indicies should match the the positions of auxiliary outcomes. The output layer and loss function for CIDNN is different from those for NN, whereas the input layer and hidden layers of CIDNN are identical.

In [ ]:
import tensorflow as tf

def CIScore(y_true, y_pred):
  CI_numer = []
  CI_numer.append(tf.square((y_pred[:, 3] - y_pred[:, 1] * y_pred[:, 0])))
  CI_numer.append(tf.square((y_pred[:, 4] - y_pred[:, 2] * y_pred[:, 0])))

  return tf.reduce_mean(CI_numer[0], axis=-1)/CI_denom[0] + tf.reduce_mean(CI_numer[1], axis=-1)/CI_denom[1] + tf.keras.losses.binary_crossentropy(y_true[:,0], y_pred[:,0])

def train_CIDNN(model, X, y, epochs, batch_size):
  if model is None:
    model = define_NN(y.shape[1])

  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
  rl = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

  model.compile(loss=CIScore, optimizer=SGD(learning_rate = 0.001, momentum=0.9), metrics=['accuracy'])
  history = model.fit(X , y, batch_size = batch_size, epochs = epochs, validation_split=0.2, callbacks=[es, rl], verbose=2)
  return model, history


## Joint-model-assisted linear/nonlinear decision rules (LDR/NLDR)

For LDR, the original X is flattened first, and then the flattened X and transformed X are concatenated for the input. For NLDR, the concatenation is performed once convolutions are done.

In [ ]:
def define_LDR(input_shape2):

  inputA = Input(shape = (28,28,1,))
  inputB = Input(shape = (input_shape2,))

  modelA = Flatten()(inputA)
  modelA = Model(inputs = inputA, outputs = modelA)

  combined = concatenate([modelA.output, inputB])
  modelB = Dense(1, activation = 'sigmoid')(combined)

  model_transf = Model(inputs = [modelA.input, inputB], outputs = modelB)
  model_transf.summary()
  return model_transf

def train_LDR(model, X, X_extracted, y, epochs, batch_size):
  
  if model is None:
    model = define_LDR(X_extracted.shape[1])

  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
  rl = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

  model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate = 0.001, momentum=0.9), metrics=['accuracy'])
  history = model.fit([X,X_extracted] , y, batch_size = batch_size, epochs = epochs, validation_split=0.2, callbacks=[es, rl], verbose=2)
  return model, history

In [ ]:
def define_NLDR(input_shape2):

  inputA = Input(shape = (28,28,1,))
  inputB = Input(shape = (input_shape2,))

  modelA = Conv2D(64, 7, input_shape = (28, 28, 1), padding='same', activation = "relu")(inputA)
  modelA = MaxPooling2D(pool_size = (2,2))(modelA)
  modelA = Conv2D(128, 5, activation = "relu", padding='same')(modelA)
  modelA = MaxPooling2D(pool_size = (2,2))(modelA)
  modelA = Flatten()(modelA)
  modelA = (Dropout(0.5))(modelA)
  modelA = Model(inputs = inputA, outputs = modelA)

  combined = concatenate([modelA.output, inputB])
  modelB = Dense(512, activation = 'relu')(combined)
  modelB = Dense(1, activation = 'sigmoid')(modelB)

  model_transf = Model(inputs = [modelA.input, inputB], outputs = modelB)
  model_transf.summary()
  return model_transf

def train_NLDR(model, X, X_extracted, y, epochs, batch_size):
  
  if model is None:
    model = define_NLDR(X_extracted.shape[1])

  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
  rl = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
  model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate = 0.001, momentum=0.9), metrics=['accuracy'])
  history = model.fit([X,X_extracted] , y, batch_size = batch_size, epochs = epochs, validation_split=0.2, callbacks=[es, rl], verbose=2)
  return model, history

# Training

We use 10-fold cross-validation for the overall procedure. 

In [ ]:
y_tgt = np.array(y_tgt)

np.random.seed(42)
fold_idx = np.random.choice(np.hstack((np.repeat(np.arange(10), int(X.shape[0]/10)), np.arange(X.shape[0] % 10))), size=X.shape[0], replace=False)

def split_data(X, y, fold_no):
  X_train, X_test, y_train, y_test = X[fold_idx != fold_no], X[fold_idx == fold_no], y[fold_idx != fold_no], y[fold_idx == fold_no]
  np.random.seed(fold_no)
  shuffle_train = np.random.choice(X_train.shape[0], X_train.shape[0], replace=False)
  shuffle_test = np.random.choice(X_test.shape[0], X_test.shape[0], replace=False)
  return X_train[shuffle_train], X_test[shuffle_test], y_train[shuffle_train], y_test[shuffle_test]

We strongly encourage you to mount Google drive with the given directory, '/content/drive' to reproduce our work easily. Also, it takes long time to save and load models from your local drive. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import keras
for fold_no in np.arange(10):

  X_train, X_test, y_aux_train, y_aux_test = split_data(X, y_aux_agg, fold_no)
  X_train, X_test, y_tgt_train, y_tgt_test = split_data(X, y_tgt, fold_no)
  y_aux_train = np.array(y_aux_train, dtype='float32')
  sample_size = y_aux_train.shape[0]
  

  # Baseline
  model_Baseline, history_Baseline = train_Baseline(model=None, X=X_train, y=y_tgt_train, epochs = 300, batch_size = 256)
  model_Baseline.save("/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_Baseline"+str(fold_no))
  keras.backend.clear_session()

  # NN
  model_NN, history_NN = train_NN(model=None, X=X_train, y=y_tgt_train, epochs = 300, batch_size = 256)
  model_NN.save("/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_NN"+str(fold_no))
  keras.backend.clear_session()

  # NN joint
  model_NN_joint, history_NN_joint =  train_NN(model =None, X=X_train, y=y_aux_train[:,0:3], epochs = 300, batch_size = 256)
  model_NN_joint.save("/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_NN_joint"+str(fold_no))
  keras.backend.clear_session()

  # CIDNN
  ## the denominator of the first term of CIScore
  CI_denom = []
  CI_denom.append((tf.square(tf.reduce_sum(y_aux_train[:,3], axis=-1)/sample_size - 
                          tf.reduce_sum(y_aux_train[:,1], axis=-1)/sample_size * 
                          tf.reduce_sum(y_aux_train[:,0], axis=-1)/sample_size)))
  CI_denom.append((tf.square(tf.reduce_sum(y_aux_train[:,4], axis=-1)/sample_size - 
                          tf.reduce_sum(y_aux_train[:,2], axis=-1)/sample_size * 
                          tf.reduce_sum(y_aux_train[:,0], axis=-1)/sample_size)))
  ## training the model  
  model_CIDNN, history_CIDNN = train_CIDNN(model=None, X=X_train, y=y_aux_train, epochs=300, batch_size = 256)
  model_CIDNN.save("/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_CIDNN"+str(fold_no))
  keras.backend.clear_session()

  # Extracting transformed features from the CIDNN
  model_CIDNN = keras.models.load_model("/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_CIDNN"+str(fold_no), compile= False)
  extraction = Model(inputs=model_CIDNN.inputs, outputs=model_CIDNN.layers[-2].output)
  extracted_features = extraction.predict(X_train)

  # LDR-CIDNN
  model_LDR_CIDNN, history_LDR_CIDNN = train_LDR(model=None, X=X_train, X_extracted= extracted_features, y=y_tgt_train, epochs=300, batch_size=256)
  model_LDR_CIDNN.save("/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_LDR_CIDNN"+str(fold_no))
  keras.backend.clear_session()

  # NLDR-CIDNN
  model_NLDR_CIDNN, history_NLDR_CIDNN = train_NLDR(model=None, X=X_train, X_extracted= extracted_features, y=y_tgt_train, epochs=300, batch_size=256)
  model_NLDR_CIDNN.save("/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_NLDR_CIDNN"+str(fold_no))
  keras.backend.clear_session()

  # Extracting transformed features from the joint model
  model_NN_joint = keras.models.load_model("/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_NN_joint"+str(fold_no), compile=False)
  extraction_ce = Model(inputs=model_NN_joint.inputs, outputs=model_NN_joint.layers[-2].output)
  extracted_features_ce = extraction_ce.predict(X_train)
  
  # LDR-NN-joint
  model_LDR_NN_joint, history_LDR_NN_joint = train_LDR(model=None, X=X_train, X_extracted= extracted_features_ce, y=y_tgt_train, epochs=300, batch_size=256)
  model_LDR_NN_joint.save("/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_LDR_NN_joint"+str(fold_no))
  keras.backend.clear_session()

  # NLDR-NN-joint
  model_NLDR_NN_joint, history_NLDR_NN_joint = train_NLDR(model=None, X=X_train, X_extracted= extracted_features_ce, y=y_tgt_train, epochs=300, batch_size=256)
  model_NLDR_NN_joint.save("/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_NLDR_NN_joint"+str(fold_no))
  keras.backend.clear_session()


# Evaluation

Evaluate models with the following metrics: AUC, accuracy, F1 score. If you do not train models above, please upload the trained models we provide. 

We provide the trained models (link: https://drive.google.com/drive/folders/1sQaV1LYvmgDbtzhhtBFMavwcJyAI_C-H?usp=sharing). You can download them or copy them to your Google drive. The following link explains how to copy the shared folder to your Google drive: https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab

Note that although warning signs might pop up when loading models, it can be ignored because we follow their guideline as they mention


In [ ]:
# if you want to load models from your local drive, please use this code
from google.colab import files
files.upload()

In [ ]:
import keras 
from keras.models import Model
from sklearn import metrics

scores = {}

for fold_no in np.arange(10):
  X_train, X_test, y_aux_train, y_aux_test = split_data(X, y_aux_agg, fold_no)
  X_train, X_test, y_tgt_train, y_tgt_test = split_data(X, y_tgt, fold_no)
  y_aux_train = np.array(y_aux_train, dtype='float32')

  model_Baseline = keras.models.load_model('/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_Baseline'+str(fold_no))
  model_NN = keras.models.load_model('/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_NN'+str(fold_no))
  model_NN_joint = keras.models.load_model('/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_NN_joint'+str(fold_no))
  model_CIDNN = keras.models.load_model('/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_CIDNN'+str(fold_no), compile=False)
  model_NLDR_CIDNN = keras.models.load_model('/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_NLDR_CIDNN'+str(fold_no))
  model_LDR_CIDNN = keras.models.load_model('/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_LDR_CIDNN'+str(fold_no))
  model_NLDR_NN_joint = keras.models.load_model('/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_NLDR_NN_joint'+str(fold_no))
  model_LDR_NN_joint = keras.models.load_model('/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/model_LDR_NN_joint'+str(fold_no))


  predicted_Baseline = model_Baseline.predict(X_test)
  predicted_NN = model_NN.predict(X_test)
  predicted_NN_joint = model_NN_joint.predict(X_test)[:,0]
  
  #CIDNN
  extraction = Model(inputs=model_CIDNN.inputs, outputs=model_CIDNN.layers[-2].output)
  predicted_extract = extraction.predict(X_test)
  predicted_LDR_CIDNN = model_LDR_CIDNN.predict([X_test, predicted_extract])
  predicted_NLDR_CIDNN = model_NLDR_CIDNN.predict([X_test, predicted_extract])
  
  #NN-joint
  extraction_NN_joint = Model(inputs=model_NN_joint.inputs, outputs=model_NN_joint.layers[-2].output)
  predicted_extract_NN_joint = extraction_NN_joint.predict(X_test)
  predicted_LDR_NN_joint = model_LDR_NN_joint.predict([X_test, predicted_extract_NN_joint])
  predicted_NLDR_NN_joint = model_NLDR_NN_joint.predict([X_test, predicted_extract_NN_joint])
  
  auc_Baseline = metrics.roc_auc_score(y_tgt_test, (predicted_Baseline).reshape(-1))
  auc_NN = metrics.roc_auc_score(y_tgt_test, (predicted_NN).reshape(-1))
  auc_NN_joint = metrics.roc_auc_score(y_tgt_test, (predicted_NN_joint).reshape(-1))
  auc_LDR_CIDNN = metrics.roc_auc_score(y_tgt_test, (predicted_LDR_CIDNN).reshape(-1))
  auc_NLDR_CIDNN = metrics.roc_auc_score(y_tgt_test, (predicted_NLDR_CIDNN).reshape(-1))
  auc_NLDR_NN_joint = metrics.roc_auc_score(y_tgt_test, (predicted_NLDR_NN_joint).reshape(-1))
  auc_LDR_NN_joint = metrics.roc_auc_score(y_tgt_test, (predicted_LDR_NN_joint).reshape(-1))

  acc_Baseline = np.mean(((predicted_Baseline > 0.5) *1).reshape(-1) == y_tgt_test)
  acc_NN = np.mean(((predicted_NN > 0.5) *1).reshape(-1) == y_tgt_test)
  acc_NN_joint = np.mean(((predicted_NN_joint > 0.5) *1).reshape(-1) == y_tgt_test)
  acc_LDR_CIDNN = np.mean(((predicted_LDR_CIDNN > 0.5) *1).reshape(-1) == y_tgt_test)
  acc_NLDR_CIDNN = np.mean(((predicted_NLDR_CIDNN > 0.5) *1).reshape(-1) == y_tgt_test)
  acc_LDR_NN_joint = np.mean(((predicted_LDR_NN_joint > 0.5) *1).reshape(-1) == y_tgt_test)
  acc_NLDR_NN_joint = np.mean(((predicted_NLDR_NN_joint > 0.5) *1).reshape(-1) == y_tgt_test)

  f1_Baseline = metrics.f1_score(y_tgt_test, ((predicted_Baseline>0.5) *1).reshape(-1))
  f1_NN = metrics.f1_score(y_tgt_test, ((predicted_NN>0.5) *1).reshape(-1))
  f1_NN_joint = metrics.f1_score(y_tgt_test, ((predicted_NN_joint>0.5) *1).reshape(-1))
  f1_LDR_CIDNN = metrics.f1_score(y_tgt_test, ((predicted_LDR_CIDNN>0.5)*1).reshape(-1))
  f1_NLDR_CIDNN = metrics.f1_score(y_tgt_test, ((predicted_NLDR_CIDNN>0.5)*1).reshape(-1))
  f1_LDR_NN_joint = metrics.f1_score(y_tgt_test, ((predicted_LDR_NN_joint>0.5)*1).reshape(-1))
  f1_NLDR_NN_joint = metrics.f1_score(y_tgt_test, ((predicted_NLDR_NN_joint>0.5)*1).reshape(-1))

  scores[fold_no] = np.array([auc_Baseline, auc_NN, auc_NN_joint, auc_LDR_NN_joint, auc_LDR_CIDNN, auc_NLDR_NN_joint, auc_NLDR_CIDNN,  
                              acc_Baseline, acc_NN, acc_NN_joint, acc_LDR_NN_joint, acc_LDR_CIDNN, acc_NLDR_NN_joint, acc_NLDR_CIDNN,  
                              f1_Baseline, f1_NN, f1_NN_joint, f1_LDR_NN_joint, f1_LDR_CIDNN, f1_NLDR_NN_joint, f1_NLDR_CIDNN])

scores_final = np.vstack(scores.values())

scores_final = pd.DataFrame(scores_final, columns = ['auc_Baseline','auc_NN', 'auc_NN_joint', 'auc_LDR_NN_joint', 'auc_LDR_CIDNN', 'auc_NLDR_NN_joint', 'auc_NLDR_CIDNN',
                                                     'acc_Baseline','acc_NN', 'acc_NN_joint', 'acc_LDR_NN_joint', 'acc_LDR_CIDNN','acc_NLDR_NN_joint', 'acc_NLDR_CIDNN',
                                                     'f1_Baseline','f1_NN', 'f1_NN_joint', 'f1_LDR_NN_joint','f1_LDR_CIDNN','f1_NLDR_NN_joint', 'f1_NLDR_CIDNN'])


scores_final.to_csv('/content/drive/MyDrive/Can_a_joint_model_assist_target_label_prediction/FashionMNIST/scores_final.csv')


In [ ]:
scores_final

,auc_Baseline,auc_NN,auc_NN_joint,auc_LDR_NN_joint,auc_LDR_CIDNN,auc_NLDR_NN_joint,auc_NLDR_CIDNN,acc_Baseline,acc_NN,acc_NN_joint,acc_LDR_NN_joint,acc_LDR_CIDNN,acc_NLDR_NN_joint,acc_NLDR_CIDNN,f1_Baseline,f1_NN,f1_NN_joint,f1_LDR_NN_joint,f1_LDR_CIDNN,f1_NLDR_NN_joint,f1_NLDR_CIDNN
0,0.929552,0.966203,0.956252,0.959012,0.967696,0.967093,0.968144,0.929500,0.944167,0.935833,0.939833,0.944667,0.945167,0.946000,0.553326,0.685446,0.647113,0.656518,0.690299,0.702800,0.700555
1,0.930776,0.971275,0.965515,0.968203,0.968640,0.969553,0.968235,0.930000,0.952000,0.945667,0.948167,0.951500,0.947667,0.950833,0.567901,0.745583,0.714035,0.721076,0.744513,0.721631,0.742358
2,0.925284,0.967267,0.964523,0.965928,0.963767,0.968008,0.963916,0.926000,0.947000,0.946333,0.948667,0.945000,0.950167,0.944333,0.537500,0.717584,0.719023,0.725000,0.700544,0.734222,0.702847
3,0.927854,0.965321,0.969731,0.969901,0.972027,0.970899,0.972726,0.924500,0.946667,0.947667,0.948500,0.952833,0.949667,0.952333,0.511327,0.706422,0.716094,0.718323,0.744354,0.726449,0.744186
4,0.919498,0.965731,0.948440,0.952932,0.961681,0.959611,0.961012,0.922500,0.947667,0.931500,0.936333,0.942833,0.939500,0.944167,0.515120,0.720641,0.641674,0.647601,0.686758,0.669700,0.694622
5,0.925954,0.970017,0.961110,0.962196,0.964220,0.963400,0.964610,0.922833,0.947000,0.940667,0.940667,0.944000,0.943667,0.943333,0.539303,0.722028,0.702341,0.685512,0.703704,0.702988,0.702797
6,0.926843,0.956773,0.964652,0.964282,0.967898,0.965658,0.968652,0.928833,0.939333,0.947667,0.947000,0.950500,0.948500,0.950167,0.545261,0.653333,0.709797,0.697719,0.721127,0.710945,0.728921
7,0.927053,0.967252,0.949013,0.955925,0.967061,0.956331,0.966829,0.925333,0.951500,0.941000,0.946167,0.951500,0.944333,0.950667,0.535270,0.727273,0.667917,0.690316,0.723647,0.689013,0.722326
8,0.927444,0.963757,0.970796,0.970722,0.972968,0.972671,0.972666,0.928333,0.942833,0.948333,0.948667,0.950500,0.949333,0.948833,0.570858,0.708085,0.739496,0.735395,0.745064,0.741497,0.738278
9,0.920852,0.962665,0.957976,0.961897,0.969732,0.965674,0.969092,0.925667,0.944167,0.938167,0.943833,0.948500,0.945333,0.948167,0.505543,0.675073,0.648341,0.668633,0.707109,0.682171,0.704091


In [ ]:
np.mean(scores_final)

auc_Baseline         0.926111
auc_NN               0.965626
auc_NN_joint         0.960801
auc_LDR_NN_joint     0.963100
auc_LDR_CIDNN        0.967569
auc_NLDR_NN_joint    0.965890
auc_NLDR_CIDNN       0.967588
acc_Baseline         0.926350
acc_NN               0.946233
acc_NN_joint         0.942283
acc_LDR_NN_joint     0.944783
acc_LDR_CIDNN        0.948183
acc_NLDR_NN_joint    0.946333
acc_NLDR_CIDNN       0.947883
f1_Baseline          0.538141
f1_NN                0.706147
f1_NN_joint          0.690583
f1_LDR_NN_joint      0.694609
f1_LDR_CIDNN         0.716712
f1_NLDR_NN_joint     0.708142
f1_NLDR_CIDNN        0.718098
dtype: float64